In [1]:
!pip install git
!pip install transformers[sentencepiece]
!pip install git+https://github.com/csebuetnlp/normalizer
!pip install nltk

ERROR: Could not find a version that satisfies the requirement git (from versions: none)
ERROR: No matching distribution found for git


  Cloning https://github.com/csebuetnlp/normalizer to c:\users\razer\appdata\local\temp\pip-req-build-0teo1e20
  Resolved https://github.com/csebuetnlp/normalizer to commit d80c3c484e1b80268f2b2dfaf7557fe65e34f321
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer 'C:\Users\RAZER\AppData\Local\Temp\pip-req-build-0teo1e20'


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer
from nltk.translate.bleu_score import sentence_bleu
import torch

# calling BanglaT5 model
model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/banglat5")
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5", use_fast=False)

In [3]:
model_name = "C:/Users/RAZER/Documents/Sartaj/BQAG/model_weights_epoch_9.pth"

In [4]:
import numpy

def get_question_answer(context):
      source_encoding=tokenizer(
        context,
        max_length=1024,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt").to(device)

      #print(source_encoding)

      generated_ids=model.generate(
          input_ids=source_encoding["input_ids"],
          attention_mask=source_encoding["attention_mask"],
          num_beams = 1,
          max_length=512,
          repetition_penalty=2.5,
          length_penalty=1.0,
          early_stopping= True,
          use_cache = True,
      )

    #print(generated_ids)

      preds=[tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) for gen_id in generated_ids]

      #return "".join(preds), generated_ids
      return "".join(preds)

In [5]:
from transformers import AdamW

# setup GPU/CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# move model over to detected device

optimizer = AdamW(model.parameters(), lr=5e-5)


#calling the checkpoint and loading the parameters of the saved model for evaluation
checkpoint = torch.load(model_name)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.to(device)


model.eval()
print()

C:\Users\RAZER\anaconda3\envs\TitanoBoa\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [29]:
!pip install glob2

  Using cached glob2-0.7.tar.gz (10 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for glob2: filename=glob2-0.7-py2.py3-none-any.whl size=9305 sha256=3ecb032ff46fb91b070dfc4d322e3c55b5e845321e21324c7ccbd5f5745c87e4
  Stored in directory: c:\users\razer\appdata\local\pip\cache\wheels\c4\50\09\61cb9fc21daef18f10394fbdc99ba8ca9bcbca50579d1f3c0f
Successfully built glob2


In [38]:
from glob import glob
import pandas

articles=[]

for i in list(glob("C:/Users/RAZER/Documents/Sartaj/new_data/*")):
    df = pandas.read_csv(i)
    df = df.dropna()
    df = df.drop(['Unnamed: 0', 'class'], axis=1)
    articles1 = list(df['article'])

    for i in range(15000):
        articles.append('c: ' + normalize(articles1[i]))

    

In [39]:
len(articles)

15000

In [40]:
import numpy

def get_answer(context):
    source_encoding=tokenizer(
        context,
        max_length=1024,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt").to(device)
  
  #print(source_encoding)
    generated_ids=model.generate(
        input_ids=source_encoding["input_ids"],
        attention_mask=source_encoding["attention_mask"],
        num_beams = 1,
        max_length=256,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping= True,
        use_cache = True)
  
  #print(generated_ids)

    preds=[tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) for gen_id in generated_ids]

    return "".join(preds)

In [42]:
sample_total = len(articles)
print('No of sample in Validation set: ', sample_total)
qs = []
ans=[]

with torch.no_grad():
    l = len(articles)
    for i in range(l):
        pred = get_answer(articles[i])
        
        x = pred.split("a:")
        if len(x)==1:
            q=x[0].replace("q:","")
            a=''
        else:
            q,a =x[0],x[1]
            q=q.replace("q:","")
        qs.append(q)
        ans.append(a)

No of sample in Validation set:  15000


In [44]:

import pandas as pd
dic = {
    "Context":articles,
    "Question":qs,
    "Answer":ans
}
df = pd.DataFrame(dic)
df.to_csv("C:/Users/RAZER/Documents/Sartaj/new_data/New_data.csv",index = False)

In [45]:
df.shape

(15000, 3)

In [47]:
df = pd.read_csv("C:/Users/RAZER/Documents/Sartaj/new_data/New_data.csv")

df.head(10)

,Context,Question,Answer
0,c: শ্রাবন্তীর 'শিকারি'র পর এবার আরেক ছবি 'বাদ...,কবে ঢাকায় আসবেন জিৎ?,৩০ জুন
1,c: টিভি অভিনেত্রী প্রত্যুষা ব্যানার্জিকে প্রায...,টিভি অভিনেত্রী প্রত্যুষা ব্যানার্জিকে কোথায় ...,ফাঁসিতে ঝুলন্ত অবস্থায়
2,c: বাস্তবের দুই বোন ডেকোটা ফ্যানিং আর এল ফ্যান...,কে এই উপন্যাসে দুই বোনের ভূমিকায় অভিনয় করবেন?,ডেকোটা ফ্যানিং; এল ফ্যাং ক্রিস্টিন হ্যানা
3,c: প্রবীণ চলচ্চিত্র অভিনেত্রী শবনম এখনও অভিনয়...,শবনমের অভিনীত সর্বশেষ সিনেমা কোনটি?,আম্মাজান
4,c: করোনাভাইরাস আতঙ্কে কাঁপছে সারা বিশ্ব। গত ড...,কবে চীনের উহান শহরে উৎপত্তির পর দুই মাসেই ১০৯...,গত ডিসেম্বরে
5,c: 'সানি লিওন যেভাবে পুরুষদের আনন্দ দেয় প্রত...,রামগোপাল ভার্মা কি সানি লিওনকে আক্রমণ করেছিলেন?,হ্যাঁ
6,c: বিনোদন ডেস্ক : মোস্তফা সরয়ার ফারুকী পরিচাল...,কে ডুব-এর অনাপত্তিপত্র বাতিল করেন?,সেন্সর বোর্ড
7,c: বিনোদন ডেস্ক : নাট্যনির্মাতা বিইউ শুভ নিজের...,বিইউ শুভ'র প্রথম চলচ্চিত্র কোনটি?,'গেম-টু'
8,c: সম্প্রতি অভিনেত্রী জ্যাকুলিন ফার্নান্ডেজ ও ...,কত সালে গানটি প্রকাশিত হয়?,১৯৭৬
9,c: অবশেষে মার্কিন পপতারকা প্রিন্স রজার্স নেলস...,প্রিন্স রজার্স নেলসনের মৃত্যু নিয়ে জল্পনা-কল...,অবশেষে
